In [19]:
import pandas as pd
import numpy as np

from pymongo import MongoClient

import re

#fixed issue with ascii encoding errors
import sys

reload(sys)
sys.setdefaultencoding("utf-8")
#TODO: countvectorizer (ignores punctuation), nltk tokenizer

In [20]:
#code to put data from mongodb named Poetry into two separate Pandas DataFrame
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY

poets = db.POET
poems = db.POEM

rawpoet = pd.DataFrame(list(poets.find()))
rawpoem = pd.DataFrame(list(poems.find()))

### clean rawpoet table. At the moment, _id, poet_positions, & posyears (position years) column values appear unusable
### drop these columns for now

In [21]:
ctodrop = ['_id', 'poet_positions', 'poet_posyears']
rawpoet.drop(ctodrop, 1,inplace = True)

In [22]:
#These are the columns that contain list of strings or strings that 
#need to be turned into one string for cleaning/extraction

ctostr = ['movement_name','poet_bio','poet_name', 'poet_basicbio', 'poet_tags']

for i in ctostr:
    rawpoet[i] = rawpoet[i].map(lambda x: u''.join(x))

In [23]:
# rawpoet.head()[:1]

###the poet's movement name tag is very important, we need to clean (remove dup tags), separate out into columns, then merge back into a clean poet table

In [24]:
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(str(x))),axis=1)
rawpoet['tags'] = rawpoet[['movement_name', 'poet_tags']].apply(lambda x: list(set(x)),axis=1)
rawpoet['tags'] = rawpoet['tags'].apply(lambda x: x.replace('-','').replace(' ','').split("\n"))
rawpoet['tags']=rawpoet['tags'].map(lambda x: ''.join(x).split(","))

In [25]:
rawpoet.tags[:10]

0    [Slam/SpokenWord]
1       [Contemporary]
2       [Objectivists]
3         [Surrealism]
4         [Surrealism]
5         [Surrealism]
6         [Surrealism]
7       [Objectivists]
8       [Objectivists]
9       [Objectivists]
Name: tags, dtype: object

In [26]:
#Split out the list of tags from the tags column and put into a data frame to then concat 
#back into a final cleaned poet dataframe
ptags = rawpoet['tags'].apply(pd.Series)
ptags.columns = ['primary_tag', 'tag2', 'tag3','tag4']
ptags.head()

,primary_tag,tag2,tag3,tag4
0,Slam/SpokenWord,NaN,NaN,NaN
1,Contemporary,NaN,NaN,NaN
2,Objectivists,NaN,NaN,NaN
3,Surrealism,NaN,NaN,NaN
4,Surrealism,NaN,NaN,NaN


##now we need to look at poet births/deaths

In [27]:
def extract_birth(text, born = True):
    """Helper function that takes a list of two strings of text, converts them into integers 
    return the either the first one or the second (if the second isn't empty; empty = 0)"""
    extracted = 0
    if str(text) != 'nan':
        if not born:
            if len(text)>1:
                extracted = int(text[1])
        else:           
            extracted = int(text[0])
    return extracted

In [28]:
#use the helper function extract_birth to fill the poet birth yr, yr died (if dead) variables
rawpoet['yr_born'] = rawpoet['poet_dob2'].apply(lambda x: extract_birth(x))
rawpoet['yr_died'] = rawpoet['poet_dob2'].apply(lambda x: extract_birth(x,False))

In [29]:
#put the poet_basicbio, which is a string, into another cleaned string column to work with
rawpoet['basic_bio']=rawpoet['poet_basicbio'].apply(lambda x: x.replace('-',''))

poet_basicbio contains a secondary source of birth and death years. It also contains where the poet was born; city, state and country. We need to extract this information.

In [30]:
#extract anything that is a number
rawpoet['birthdeath'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'\d+', x))

#extract non-numeric characters
rawpoet['birthplace'] = rawpoet['basic_bio'].map(lambda x: re.findall(r'[^\d]+', x))
rawpoet['birthplace'] = rawpoet['birthplace'].apply(lambda x: u''.join(x))

In [31]:
def extract_bio(text, part = 'city'):
    """Helper function to extract city, state, and country string variables from the birthplace column"""
    
    split_text_blob = text.split(',')
    
    # based on the data found in the column, the second item as delimited by the comma is the country
    # where the poet represents/is from, the first item in the column is the city state, separated by spaces
    
    citystate = split_text_blob[0].strip().split(' ')
    
    city = []
    state = ''
    country = ''
    
    for word in citystate:
        #state is two chars long
        if len(word) == 2:
            state = word
        #city names are longer than state abbreviation, could also contain 
        #county or neighborhood if poet is non-US based
        if len(word) > 2:
            city.append(word)
            
    if len(word)>1:
        country = word[1]
    
    if part == 'city':
        return ' '.join(city).strip()
    elif part == 'state':
        return state.strip()
    else:
        return country.strip()

In [32]:
rawpoet['city'] = rawpoet['birthplace'].map(lambda x: extract_bio(x))
rawpoet['state'] = rawpoet['birthplace'].map(lambda x: extract_bio(x, 'state'))
rawpoet['country'] = rawpoet['birthplace'].map(lambda x: extract_bio(x, 'country'))

In [33]:
#Now the data is more or less ready to put in cleaned dataframe of poet information
#rawpoet.columns

coltokeep= ['poet_name','poet_url','poet_poems_url','tags','poet_bio','yr_born','yr_died','city','state','country']

poetdf = pd.concat([rawpoet[coltokeep],ptags],axis=1)

In [34]:
#Not sure if I want to keep these variables calculated from the birth and death information about each poet
#using them for now
#It's 2015 now, so use this to calculate current age if poet hasn't died

poetdf['dead'] = np.where(poetdf['yr_died'] > 0,1,0)
poetdf['current_age'] = np.where(poetdf['dead'] == 0, (2015- poetdf['yr_born']), 0)
poetdf['age_dead'] = np.where(poetdf['dead'] == 1, (poetdf['yr_died']-poetdf['yr_born']), 0)
poetdf['current_age'] = np.where(poetdf['current_age'] == 2015, 0,poetdf['current_age'])

In [35]:
#Combine cleaned poet dataframe and poem data frame based on the foreign key identifier, poet_poems_url
# rawpoem.head()
poemdf = rawpoem[['poet_poems_url','poem_title', 'poem_yrpub', 'poem_text', 'poem_link']]
resultdf = pd.merge(poetdf, poemdf, how='inner', on=['poet_poems_url'])

In [36]:
#Save the results to a pickle file. Alternatively, can also save to a collection in the same POETRY database

resultdf.to_pickle('resultdf.pkl')

Variables:

* year published
* total number of words
* avg words per new line
* total number of stanzas
* avg newlines per stanza
* if first word of each line starts with capital letter
* number of words with capital letters
* number of commas
* commas per line
* number of periods
* number of exclamations
* number of pronoun references
* if i, my is used bool
* type token ratio
* concrete words/words overall
* abstract/words overall
* number of named entities/total words
* alliteration
* number of sensory words
* topic modeling - political

In [37]:
resultdf.columns
# rawpoem.columns

Index([     u'poet_name',       u'poet_url', u'poet_poems_url',
                 u'tags',       u'poet_bio',        u'yr_born',
              u'yr_died',           u'city',          u'state',
              u'country',    u'primary_tag',           u'tag2',
                 u'tag3',           u'tag4',           u'dead',
          u'current_age',       u'age_dead',     u'poem_title',
           u'poem_yrpub',      u'poem_text',      u'poem_link'],
      dtype='object')